## Problem 1: Sample Recolection
<br> <div style="text-align: justify"> 
The first step to applying the Policy Iteration Algorithm to this problem will be coding all of the initial conditions of the problem. We start by coding the time steps, the possible number of samples and the possible actions to be taken. Due to the fact that this is a finite, *non-stationary* MDP, combinations of time steps and number of samples will be considered the states of the problem. For example, 1250 samples at 500m of immersion will be an state and 1250 samples at 400m of immersion will be another state. 
</div> 

In [1]:
### Time steps of decision
T = [1000, 900, 800, 700, 600, 500, 400, 300, 200, 100, 0]

### Samples
Samples = [0, 250, 500, 750, 1000, 1250, "Succes"]

### Space of States
S_t = []
for S in [0, 250, 500, 750]:
    S_t.append((1000,S))
for T in [900, 800, 700, 600, 500, 400, 300, 200, 100, 0]:
    for S in Samples:
        S_t.append((T,S))

### Actions 
# CM for Calculated Maneuver
# IM for improvised Maneuver 
A = ["CM", "IM"]

<br> <div style="text-align: justify"> 
Next, transition probabilities will be coded. As mention before, this probabilities will indicate the stochastic nature of the problem on the transitions between states. As the states contain steps of decision and number of samples, this information will be taken into account in the transition probabilities. Also, the probabilities depend on the action that is performed. Therefore, there will be to dictionaries of probabilites, one for Calculated Maneuvers and one for Improvised Maneuvers. At the end, both of this dictionaries will be consolidated in one dictionary. 
</div> 

In [2]:
### Transition probabilities
## Transition Probabilites will be defined as dictionaries
# Transition Probabilities for Calculated Maneuvers
p_CM = {}
for s_t in S_t:
    for s_tplus1 in S_t:
        if s_tplus1[0] == s_t[0] - 100:
            if s_t[1] != "Succes" and s_tplus1[1] != "Succes":
                if s_tplus1[1] == s_t[1] + 250 and s_t[1] < 1250:
                    p_CM[s_t, s_tplus1] = 0.3
                elif s_tplus1[1] == s_t[1] + 500 and s_t[1] < 1000:
                    p_CM[s_t, s_tplus1] = 0.2
                elif s_tplus1[1] == s_t[1] + 750 and s_t[1] < 750:
                    p_CM[s_t, s_tplus1] = 0.1
                elif s_tplus1[1] == s_t[1] and s_t[1] != "Succes":
                    p_CM[s_t, s_tplus1] = 0.4
                else: 
                    p_CM[s_t, s_tplus1] = 0
            elif s_tplus1[1] == "Succes" and s_t[1] == 1000:
                p_CM[s_t, s_tplus1] = 0.3
            elif s_tplus1[1] == "Succes" and s_t[1] == 1250:
                p_CM[s_t, s_tplus1] = 0.6
            elif s_tplus1[1] == "Succes" and s_t[1] == 750:
                p_CM[s_t, s_tplus1] = 0.1
            elif s_tplus1[1] == s_t[1] and s_t[1] == "Succes":
                p_CM[s_t, s_tplus1] = 1
            else: 
                p_CM[s_t, s_tplus1] = 0
        else:
            p_CM[s_t, s_tplus1] = 0
    
# Transition Probabilites for Improvised Maneuvers
p_IM = {}
for s_t in S_t:
    for s_tplus1 in S_t:
        if s_tplus1[0] == s_t[0] - 100:
            if s_t[1] != "Succes" and s_tplus1[1] != "Succes":
                if s_tplus1[1] == s_t[1] + 500 and s_t[1] < 1000:
                    p_IM[s_t, s_tplus1] = 0.5
                elif s_tplus1[1] == s_t[1] and s_t[1] <= 1250:
                    p_IM[s_t, s_tplus1] = 0.5
                else: 
                    p_IM[s_t, s_tplus1] = 0
            elif s_tplus1[1] == "Succes" and (s_t[1] == 1000 or s_t[1] == 1250):
                p_IM[s_t, s_tplus1] = 0.5
            elif s_tplus1[1] == s_t[1] and s_t[1] == "Succes":
                p_IM[s_t, s_tplus1] = 0.1
            else: 
                p_IM[s_t, s_tplus1] = 0
        else:
            p_IM[s_t, s_tplus1] = 0

# Consolidation of probabilities
pTrans = {"CM":p_CM, "IM": p_IM}

<br> <div style="text-align: justify"> 
Finally, to finish the basic definition of the problem, the rewards must be defined. As mentioned in the problem's definition, the only reward will be if the mission has succeded at the end of the immersion. Meaning, a reward of 1 will be recieved only at state ("Succes", 0).
</div> 

In [3]:
### Rewards
# Reward r(s_t)
r = {}
for s_t in S_t:
    r[s_t] = 0

r[(0,"Succes")] = 1

## Policy Iteration

<br> <div style="text-align: justify"> 
The first step of the Policy Iteration algorithm is initializing arbitrarily the Policy's Value for each state and the Policy. As opose to the policy used in *Policy Evaluation*, this one will be a *deterministic* policy. This means, that the action taken in a particular state will be certain.
</div> 

In [4]:
### Arbitrary Policy π
pi = {}
# The initialization will be taking calculated maneuvers in all states
for s_t in S_t:
    if s_t[0] != 0:
        pi[s_t] = "CM"

### Policy's Value
# Initializing dictionary V for the value of the policy for each state in 0.5 (arbitrarily)
V = {}
for s_t in S_t:
    V[s_t] = 0.5
for samples in [0, 250, 500, 750, 1000, 1250, "Succes"]:
    V[(0,samples)] = 0

<br> <div style="text-align: justify"> 
Then, the loop is coded. The threshold $\theta$ will be the same for all iterations. Then, a boolean variable is declared, this variable will indicate if the current policy have or not a change that will improve it's performance. It is initialized in False so the loop can start. Inside the loop, first there will be a **Policy Iterative Evaluated** just as the one implemented previously. The only change is the adjustments for the *deterministic* and not *stochastic* policies. Afterwards, the **Policy Improvement** will be performed. This loop will continue until there is no improvements over the policy. Additionaly, a variable is declared to track how many iterations were made before achieving the optimal policy.
</div>

In [5]:
### Policy Iteration
# Threshold theta indicates the desired accuracy of the finded value throught iterations
theta = 0.005

# Policy_Stable indicates if the policy doesn't change in the current iteration
Policy_Stable = False

# Variable tracking the number of iterations
num_iter = 0

# The loop will be repeated until there is no improvement
while not Policy_Stable:
    
    #### Policy Evaluation
    # Initializing delta in 10 so the loop can start
    delta = 10
    # Iterations
    while delta > theta:
        delta = 0
        for s_t in S_t:
            if s_t[0] != 0:
                v = V[s_t]
                value = 0
                for a in A:
                    if pi[s_t] == a:
                        for s_tplus1 in S_t:
                            value += pTrans[a][s_t, s_tplus1] * (r[s_tplus1] + V[s_tplus1]) 
                V[s_t] = value
                delta = max(delta, abs(v - V[s_t]))

    #### Policy Improvement
    Policy_Stable = True
    for s_t in S_t:
        if s_t[0] != 0:
            old_action = pi[s_t]
            arg = 0
            argmax = "Mission Failed"
            for a in A:
                value = 0
                for s_tplus1 in S_t:
                    value += pTrans[a][s_t, s_tplus1] * (r[s_tplus1] + V[s_tplus1]) 
                if value > arg:
                    arg = value
                    argmax = a
            if old_action != argmax:
                Policy_Stable = False
                pi[s_t] = argmax
    
    ### Update the number of iterations
    num_iter += 1

print(f"Number of iterations before achieving optimality: {num_iter}")

Number of iterations before achieving optimality: 4


Finally, the optimal policy has to be evaluated since in the las iteration the optimal policy was achieved but not evaluated.

In [6]:
#### Policy Evaluation for Optimal Policy
# Initializing delta in 10 so the loop can start
delta = 10
# Iterations
while delta > theta:
    delta = 0
    for s_t in S_t:
        if s_t[0] != 0:
            v = V[s_t]
            value = 0
            for a in A:
                if pi[s_t] == a:
                    for s_tplus1 in S_t:
                        value += pTrans[a][s_t, s_tplus1] * (r[s_tplus1] + V[s_tplus1]) 
            V[s_t] = value
            delta = max(delta, abs(v - V[s_t]))

Policy Iteration has been completed. Now, specific values and decisions of the optimal policy can be found.

In [7]:
print("The optimal decision for the first 400 meters of immersion with 0 samples are:")
for t in [1000, 900, 700, 800]:
    print(f"For {t}m: {pi[(t,0)]}")
print("(CM for Calculated Maneuver and IM for Improvised Maneuver)")

print("\n")
print("The probabilities of succes, following the optimal policy, at the middle of the immersion are:")
for samples in [0, 250, 500, 750, 1000, 1250]:
    print(f"If there are {samples} samples, the probablity is {round(V[(500,samples)],3)}")

The optimal decision for the first 400 meters of immersion with 0 samples are:
For 1000m: CM
For 900m: CM
For 700m: IM
For 800m: IM
(CM for Calculated Maneuver and IM for Improvised Maneuver)


The probabilities of succes, following the optimal policy, at the middle of the immersion are:
If there are 0 samples, the probablity is 0.501
If there are 250 samples, the probablity is 0.652
If there are 500 samples, the probablity is 0.812
If there are 750 samples, the probablity is 0.901
If there are 1000 samples, the probablity is 0.969
If there are 1250 samples, the probablity is 0.99


## Problem 2: Aqueduct
<br> <div style="text-align: justify"> 
The same iterative process will be performed in this problem with some distinctions due to the characteristics of the problem. First, this is a *infinite* MDP, meaning the time steps have no valuable information. Considering this, states will contain only the liters of trash. Second, a discount factor will be applyied as stated in the description of the algorithm. On this terms, the basics of the problem will be coded in the next section: States, Transition Probabilities, Costs and the Discount Factor
</div> 

In [8]:
### States
S = list(range(3,11))

### Decisions
A = ["Send", "Don't Send"]

### Transition probabilities
# Send the cleaning team
p_SEND = {}
for s_t in S:
    for s_tplus1 in S:
        if s_tplus1 == 3:
            p_SEND[s_t, s_tplus1] = 1
        else:
            p_SEND[s_t, s_tplus1] = 0

# Dont's send the cleaning team 
p_DONTS = {}
for s_t in S:
    for s_tplus1 in S:
        if s_tplus1 >= s_t:
            p_DONTS[s_t, s_tplus1] = 1/(11-s_t)
        else:
            p_DONTS[s_t, s_tplus1] = 0

# Consolidation of probabilities
pTrans = {"Send": p_SEND, "Don't Send": p_DONTS}

### Costs
c = {}
for s_t in S:
    c[s_t, "Send"] = 12 + s_t * 0.01 * 200
    c[s_t, "Don't Send"] =  s_t * 0.01 * 200

### Discount Factor 
gamma = 0.95

## Policy Iteration

<br> <div style="text-align: justify"> 
The first step of the Policy Iteration algorithm is initializing arbitrarily the Policy's Value for each state and the Policy. The Policy's Value will be arbitrarily set to 300 million pesos for all states. The Policy, on the other hand, will be initialized as sending the team in every state.
</div> 

In [9]:
### Arbitrary policy π
pi = {}
# The initialization will be sending the team in all states
for s_t in S:
    pi[s_t] = "Send"


### Policy's Value
V = {}
# Initializing dictionary V for the value of the policy for each state in 300 (arbitrarily)
for s_t in S:
    V[s_t] = 300

Afterwards, **Policy Iteration** can be performed. The parameters for the loop will be the same as the ones used in the Problem 1. 

In [10]:
### Policy Iteration
# Threshold theta indicates the desired accuracy of the finded value throught iterations
theta = 0.005

# Policy_Stable indicates if the policy doesn't change in the current iteration
Policy_Stable = False

# Variable tracking the number of iterations
num_iter = 0

# The loop will be repeated until there is no improvement
while not Policy_Stable:
    
    #### Policy Evaluation
    # Initializing delta in 10 so the loop can start
    delta = 10
    # Iterations
    while delta > theta:
        delta = 0
        for s_t in S:
                v = V[s_t]
                value = 0
                for a in A:
                    if pi[s_t] == a:
                        for s_tplus1 in S:
                            value += pTrans[a][s_t, s_tplus1] * (c[s_t,a] + gamma * V[s_tplus1]) 
                V[s_t] = value
                delta = max(delta, abs(v - V[s_t]))

    #### Policy Improvement
    Policy_Stable = True
    for s_t in S:
        old_action = pi[s_t]
        arg = 500
        argmin = ""
        for a in A:
            value = 0
            for s_tplus1 in S:
                value += pTrans[a][s_t, s_tplus1] * (c[s_t, a] + gamma * V[s_tplus1]) 
            if value < arg:
                arg = value
                argmin = a
        if old_action != argmin:
            Policy_Stable = False
            pi[s_t] = argmin
    
    ### Update the number of iterations
    num_iter += 1

print(f"Number of iterations before achieving optimality: {num_iter}")

Number of iterations before achieving optimality: 3


Finally, the evaluation of the Optimal Policy will be performed.

In [11]:
#### Optimal Policy Evaluation
# Initializing delta in 10 so the loop can start
delta = 10
# Iterations
while delta > theta:
    delta = 0
    for s_t in S:
        v = V[s_t]
        value = 0
        for a in A:
            if pi[s_t] == a:
                for s_tplus1 in S:
                    value += pTrans[a][s_t, s_tplus1] * (c[s_t,a] + gamma * V[s_tplus1]) 
        V[s_t] = value
        delta = max(delta, abs(v - V[s_t])) 

Now that Policy Improvement is done, the decision to make and expected cost for every state can be found. This, when following the optimal policy, of course. 

In [12]:
print("The optimal decision to make is:")
for s in S:
    print(f"- If there are {s} liters of trash, {pi[s]} the Team")
    
print("\n")
print("The expected cost for every state is:")
for s in S:
    print(f"- When there are {s} liters of trash: {round(V[s],3)} Million Pesos")

The optimal decision to make is:
- If there are 3 liters of trash, Don't Send the Team
- If there are 4 liters of trash, Don't Send the Team
- If there are 5 liters of trash, Don't Send the Team
- If there are 6 liters of trash, Send the Team
- If there are 7 liters of trash, Send the Team
- If there are 8 liters of trash, Send the Team
- If there are 9 liters of trash, Send the Team
- If there are 10 liters of trash, Send the Team


The expected cost for every state is:
- When there are 3 liters of trash: 298.468 Million Pesos
- When there are 4 liters of trash: 301.743 Million Pesos
- When there are 5 liters of trash: 304.923 Million Pesos
- When there are 6 liters of trash: 307.545 Million Pesos
- When there are 7 liters of trash: 309.545 Million Pesos
- When there are 8 liters of trash: 311.545 Million Pesos
- When there are 9 liters of trash: 313.545 Million Pesos
- When there are 10 liters of trash: 315.545 Million Pesos
